# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
#print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    print(file_path_list[:3])

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv']
['/home/workspace/event_data/.ipynb_checkpoints/2018-11-15-events-checkpoint.csv', '/home/workspace/event_data/.ipynb_checkpoints/2018-11-22-events-checkpoint.csv', '/home/workspace/event_data/.ipynb_checkpoints/2018-11-08-events-checkpoint.csv']


In [3]:

path = os.getcwd() # use your path

all_files = glob.glob(path + '/event_data' +"/*.csv")

frame=None
frame = pd.DataFrame()
for filename in all_files:
    df1 = pd.read_csv(filename,header=0)
    frame = pd.concat([frame,df1])
print(frame.shape)
frame.head()


(8056, 17)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540510e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1.543280e+12,88.0
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540510e+12,961,Woodstock Inprovisation,200,1.543280e+12,88.0
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540510e+12,961,Majesty (LP Version),200,1.543280e+12,88.0
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1.543280e+12,38.0
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1.543280e+12,38.0


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
from time import time
a = time()

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

b = time()-a
print(b)

0.0939931869506836


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
frame = pd.read_csv('event_datafile_new.csv')
frame.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 

try:
    query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor': 1}"""
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
frame.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


# Query 1

In [11]:
# Query 1
# Create Table music_app_history
try:
    create = """CREATE TABLE IF NOT EXISTS music_app_history (sessionId int, itemInSession int, artist text, 
    song text,length float, PRIMARY KEY (sessionId, itemInSession))"""
    session.execute(create)

except Exception as e:
    
    print(e)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    query1 = "SELECT artist, song, length FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
    rows = session.execute(query1)
    
    for row in rows:
        #print(row.artist, row.song, row.length)
        #print(row)
        print(row.artist, row.song, row.length)
except Exception as e:
    print(e)
    
#row.sessionid, row.iteminsession,

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2

In [14]:
# Query 2: 
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session_history"
query = query +  "(artist text, song text, firstName text,lastName text, userid int, sessionId int, itemInsession int,  PRIMARY KEY (userid, sessionid, itemInSession))"
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query2 = """INSERT INTO user_session_history (artist,firstname, lastname,  song, userid, iteminsession, sessionid)
        
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        #query2 = INSERT INTO table2 (artist, firstName, lastName, userId, song, itemInSession, sessionId)

        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (line[0], line[1], line[4], line[9], int(line[10]), int(line[3]), int(line[8])))    
                   

In [16]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    query="""SELECT artist, song, firstname, lastname FROM user_session_history WHERE userid=10 and sessionid=182"""
    rows = session.execute(query)
    for i in rows:
        print(i.artist, i.firstname, i.lastname, i.song,)

        
except Exception as e:
    print(e)
#  i.userid, i.sessionid  

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


# Query 3

In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_song_history "
query = query + "(song text,userId int, firstname text, lastname text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)           

                    

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        ##  Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_history ( song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ##  Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [19]:
## Add in the SELECT statement to verify the data was entered into the table 
query = "SELECT firstname,lastname FROM user_song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions

In [21]:
try:
    session.execute("DROP TABLE IF EXISTS sparkify.music_app_history")
    session.execute("DROP TABLE IF EXISTS sparkify.user_session_history")
    session.execute("DROP TABLE IF EXISTS sparkify.user_song_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()